In [11]:
from transformers import AutoTokenizer, T5Model, T5ForConditionalGeneration

In [12]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [13]:
model = T5Model.from_pretrained("t5-small")

input_ids = tokenizer("Studies have been shown that owning a dog is good for you",
                      return_tensors="pt").input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

# preprocess: Prepend decoder_input_ids with start token which is pad token for T5Model.
# This is not needed for torch's T5ForConditionalGeneration as it does this internally using labels arg.
decoder_input_ids = model._shift_right(decoder_input_ids)   # Teacher Forcing

# forward pass
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
# Sequence of hidden-states at the output of the last layer of the decoder of the model.
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states.shape)  # [batch_size, sequence_length, hidden_size]

torch.Size([1, 4, 512])


In [14]:
model_cg = T5ForConditionalGeneration.from_pretrained("t5-small")

# training
input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids
labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids

outputs = model_cg(input_ids=input_ids, labels=labels)
loss = outputs.loss  # Language modeling loss.
# Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
logits = outputs.logits
print(loss)
print(logits.shape)  # batch_size, sequence_length, config.vocab_size

tensor(3.7837, grad_fn=<NllLossBackward0>)
torch.Size([1, 7, 32128])


In [15]:
# inference
input_ids = tokenizer("summarize: studies have shown that owning a dog is good for you", return_tensors="pt").input_ids  # Batch size 1
outputs = model_cg.generate(input_ids)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# studies have shown that owning a dog is good for you.

tensor([[   0, 2116,   43, 2008,   24,  293,   53,    3,    9, 1782,   19,  207,
           21,   25,    3,    5,    1]])
studies have shown that owning a dog is good for you.
